In [1]:
import json
import pandas as pd
import numpy as np

In [139]:
def main(position):
    dfa = dataframe_dt(position, version_list[0])
    dfb = dataframe_dt(position, version_list[1])
    frames = [dfa, dfb]
    df = pd.concat(frames, ignore_index=True)
    df.Value = df.Value.div(max(df.Value.to_list()))
    df = df.sort_values(by=['Value'], ascending=False)
    return df

def import_scraped_data(time, position, version):
    filename = "../scraped_data/%s_%s_%s.json" % (time, position, version)
    with open(filename, encoding='utf-8') as json_data:
        d = json.load(json_data)
        json_data.close()
    return pd.DataFrame(d)

def clean_dataframe(df):
    df = df.loc[:, ["Name", "Rating", "PS4", "Games"]]
    df.rename(columns={'PS4':'Price'}, inplace=True)
    df.Name = df.Name.str.replace(r"\(.*\)","")
    df.Rating = pd.to_numeric(df.Rating)
    df.Games = pd.to_numeric(df.Games.str.replace(",",""))
    df.Price = (df.Price.replace(r'[KM]+$', '', regex=True).astype(float) * \
                    df.Price.str.extract(r'[\d\.]+([KM]+)', expand=False)
                    .fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int))
    df.Price = df.Price.astype("int64")
    return df

def value(games, price):
    P = np.array(price.to_list())
    N = np.array(games.to_list())
    V = (A + (1 - A) / (1 + np.exp(-a * (P - b)))) * (B * (c * N + d))
    return V

def dataframe_dt(position, version):
    df0 = import_scraped_data(time_list[0], position, version)
    df1 = import_scraped_data(time_list[1], position, version)
    df0 = clean_dataframe(df0)
    df1 = clean_dataframe(df1)
    dn = np.zeros(len(df1.index))
    i = -1
    j = -1
    J = np.zeros(len(df1.index))
    for index_df1, row_df1 in df1.iterrows():
        i += 1
        j = -1
        n1 = row_df1["Name"]
        r1 = row_df1["Rating"]
        g1 = row_df1["Games"]
        p1 = row_df1["Price"]
        if p1 == 0:
            row_df1["Price"] = 1
            df1.iloc[i] = row_df1
            p1 = 1
        for index_df0, row_df0 in df0.iterrows():
            j += 1
            n0 = row_df0["Name"]
            r0 = row_df0["Rating"]
            g0 = row_df0["Games"]
            p0 = row_df0["Price"]
            if J[j] == 1:
                continue
            if n1 == n0 and r1 == r0:
                if g0 > g1 or abs((p1-p0)/p1) > 0.5:
                    continue
#                 print("%s %s %s %s - %s %s %s %s" % (n1, r1, g1, p1, n0, r0, g0, p0))
                dn[i] = g1 - g0
                J[j] = 1    
                break
    df1.Games = dn.astype(int)
    df1["Value"] = value(df1.Games, df1.Price)
    return df1

In [135]:
A = 0.6
a = 3e-5 # [price]
b = 7.5e4 # [-]
B = 1
c = 9.5e-6 # [games]
d = 0   # [-]

version_list = ["all_nif", "all_specials"]
position_list = ["GK", "RB,RWB", "LB,LWB", "CB", "CDM,CM,CAM", "RM,RW,RF", "LM,LW,LF", "CF,ST"]
time_list = ["2019121307", "2019121606"]

In [140]:
df = main(position_list[7])
df.head(60)

,Name,Rating,Price,Games,Value
3,Kylian Mbappé,89,767000,146120,1.000000
1,Heung Min Son,87,359000,137784,0.942876
30,Wissam Ben Yedder,85,300000,136543,0.934021
8,Aubameyang,88,110000,106361,0.652426
33,Wissam Ben Yedder,85,190000,88811,0.600315
15,Cristiano Ronaldo,93,1050000,84757,0.580051
14,Robert Lewandowski,89,44000,109006,0.532027
41,Sadio Mané,89,545000,76015,0.520223
10,Sergio Agüero,89,88000,88335,0.506912
16,Harry Kane,89,43000,92443,0.449658


In [ ]:
# Bench: Ben Yedder, de Jong, Martial
# ---alt
#  - sane




# Varane [Militao] 
# Kante [Sissoko] 1000
# Salah [Dembele]
# Aubameyang [Lucas] 1335
# Mane [Sane] 1517